In [1]:
import dask.distributed as dd
from dask.distributed import Client, LocalCluster, progress
from dask_jobqueue import PBSCluster
from dask.distributed import get_worker
import os

In [2]:
# create the user directory if it doesn't already exist
! mkdir -p /scratch/vp91/$USER

In [3]:
# set the path 
user = os.getenv('USER', 'default value')
path = '/scratch/vp91/'+user
print(path)

/scratch/vp91/jxj900


In [4]:
# The jupyter notebook is launched from your $HOME directory.
# Change the working directory the user directory under /scratch/vp91
os.chdir(os.path.expandvars(path))

In [5]:
# Make sure the python we use is from the venv
os.environ['DASK_PYTHON'] = '/scratch/vp91/Training-Venv/parallel_python/bin/python3'

In [10]:
# Make sure all the modules are loaded.
# It is essential that we use the same python and library for all aspects of dask
# If we dont activate the venv then the workers may have a different versions of libraries
setup_commands = ["module load python3/3.11.0", "module load cmake/3.16.2", "source /scratch/abc/Training-Venv/parallel_python/bin/activate"]

In [11]:
# Gadi use custom PBS directives
# So some of the default values to launch a PBS job through Dask call will not work in Gadi
# Any directive specific to gadi should be mentioned here.
# refer : https://opus.nci.org.au/display/Help/Gadi+Quick+Reference+Guide
extra = ['-q normal',
         '-P vp91', 
         '-l ncpus=48', 
         '-l mem=192GB']

In [12]:
# walltime: Walltime for each worker job.
# cores: Total number of cores per job.
# shebang: Path to desired interpreter for your batch submission script.
# job_extra_directives: List of other PBS options. Each option will be prepended with the #PBS prefix.
# local_directory: Dask worker local directory for file spilling.
# job_directives_skip: Directives to skip in the generated job script header. Directives lines containing 
#                      the specified strings will be removed. Directives added by job_extra_directives 
#                      won’t be affected.
# interface: Network interface like ‘eth0’ or ‘ib0’. This will be used both for the Dask scheduler and 
#            the Dask workers interface
# job_script_prologue: Commands to add to script before launching worker
# python: Python executable used to launch Dask workers. Defaults to the Python that is submitting these jobs



cluster = PBSCluster(walltime="00:20:00", 
                     cores=48, 
                     memory="192GB",
                     shebang='#!/usr/bin/env bash',
                     job_extra_directives=extra, 
                     local_directory='$TMPDIR', 
                     job_directives_skip=["select"], 
                     interface="ib0",
                     job_script_prologue=setup_commands,
                     python=os.environ["DASK_PYTHON"])

/scratch/vp91/Training-Venv/parallel_python/lib/python3.11/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 42625 instead
  warnings.warn(
/scratch/vp91/Training-Venv/parallel_python/lib/python3.11/site-packages/distributed/scheduler.py:6206: UserWarning: Scheduler already contains a plugin with name websocket; overwriting.
  warnings.warn(
2024-12-05 14:11:37,167 - bokeh.server.protocol_handler - ERROR - error handling message
 message: Message 'PULL-DOC-REQ' content: {} 
 error: SerializationError("can't serialize <class 'function'>")
Traceback (most recent call last):
  File "/scratch/vp91/Training-Venv/parallel_python/lib/python3.11/site-packages/bokeh/server/protocol_handler.py", line 94, in handle
    work = await handler(message, connection)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/scratch/vp91/Training-Venv/parallel_python/lib/python3.11/site-packages/bokeh/

In [13]:
print(cluster.job_script())

#!/usr/bin/env bash

#PBS -N dask-worker
#PBS -l walltime=00:20:00
#PBS -q normal
#PBS -P vp91
#PBS -l ncpus=48
#PBS -l mem=192GB
module load python3/3.11.0
module load cmake/3.16.2
source /scratch/abc/Training-Venv/parallel_python/bin/activate
/scratch/vp91/Training-Venv/parallel_python/bin/python3 -m distributed.cli.dask_worker tcp://10.6.42.31:40023 --name dummy-name --nthreads 6 --memory-limit 22.35GiB --nworkers 8 --nanny --death-timeout 60 --local-directory $TMPDIR --interface ib0



In [14]:
# create a cluster with 2 nodes
cluster.scale(jobs=2)

In [19]:
# Verify the workers have been allocated as expected
!qstat

Job id                 Name             User              Time Use S Queue
---------------------  ---------------- ----------------  -------- - -----
130162072.gadi-pbs     sys-dashboard-s* jxj900            00:00:27 R normal-exec     
130163622.gadi-pbs     dask-worker      jxj900            00:00:00 R normal-exec     
130163623.gadi-pbs     dask-worker      jxj900            00:00:00 R normal-exec     


In [ ]:
cluster

In [20]:
# create the client
client = Client(cluster)

In [21]:
client

<Client: 'tcp://10.6.42.31:40023' processes=16 threads=96, memory=357.60 GiB>

cluster.get_logs()

In [22]:
# A simple test function
def slow_increment(x):
    return x+1

In [24]:
# Submit the work to the Dask cluster
futures = client.submit(slow_increment, 5000)

In [25]:
futures

<Future: finished, type: int, key: slow_increment-f2598e5c0f206b203b55def0a4c8c287>

In [26]:
futures.result()

5001

In [29]:
import dask.dataframe as df

ddf = df.read_csv(
    os.path.join("distributed-computing-in-dask/data", "nycflights", "*.csv"),
    dtype={"TailNum": str, "CRSElapsedTime": float, "Cancelled": bool},
)

In [30]:
len(ddf)

2611892